# Pipeline to Train and Test Networks

### Paths & Library Imports

In [ ]:
from os.path import isfile, exists
from paths import RETOUCH_PATH, IMAGES_PATH

## Segmentation

### Train-Test Split

Create Train-Test Split for the Fluid Segmentation Task

In [ ]:
from init.folds_split import k_fold_split_segmentation

k = 5

if not (isfile(path="splits\segmentation_train_splits.csv") or isfile(path="splits\segmentation_test_splits.csv")):
    k_fold_split_segmentation(k=k, folders_path=RETOUCH_PATH)
else:
    print("Split already exists. To create a new one please delete the old files.")


### Images Reading and Saving

OCT Volumes Reading and Saving for Segmentation Task

In [ ]:
from init.read_oct import save_segmentation_oct_as_tiff

if not ((exists(IMAGES_PATH + "\\OCT_images\\segmentation\\slices\\int32")) and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\slices\\uint8"))):
    save_segmentation_oct_as_tiff(oct_folder=RETOUCH_PATH, save_folder=IMAGES_PATH)
else:
    print("Images have already been extracted. To extract them again, please delete the folder with the images.")

# ETA: 2m29s

OCT Masks Reading and Saving for Segmentation Task

In [ ]:
from init.read_oct import save_segmentation_mask_as_tiff

if not ((exists(IMAGES_PATH + "\\OCT_images\\segmentation\\masks\\int8")) and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\masks\\uint8"))):
    save_segmentation_mask_as_tiff(oct_folder=RETOUCH_PATH, save_folder=IMAGES_PATH)
else:
    print("Masks have already been extracted. To extract them again, please delete the folder with the images.")

# ETA: 3m4.4s

ROI Masks Extraction for Segmentation Task

In [ ]:
from init.patch_extraction import extractROIMasks

if not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\slices\\int32") and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\slices\\uint8"))):
    print("First, the images must be extracted from the OCT volumes.")
elif not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\masks\\int8") and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\masks\\uint8"))):
    print("First, the masks must be extracted from the OCT volumes.")
elif not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi")):
    extractROIMasks(oct_path=RETOUCH_PATH , folder_path=IMAGES_PATH, threshold=1e-2)
else:
    print("Patches have already been extracted. To extract them again, please delete the folder that contains the extracted ROI masks.")

# ETA: 8h46m01s

### Patch Extraction and Saving

Not required to run the project, just to check what is being done and if it is being done correctly

Patches Extraction for 2D Segmentation Task

In [ ]:
from init.patch_extraction import extractPatches

if not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi\\int8") and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi\\uint8"))):
    print("First, the ROI masks must be extracted from the OCT volumes.")
elif not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\patches\\2D")):
    extractPatches(IMAGES_PATH, patch_shape=(256,128), n_pos=12, n_neg=2, pos=1, neg=0)
else:
    print("Patches have already been extracted. To extract them again, please delete the folder that contains the extracted patches.")

Patches Extraction for 2.5D Segmentation Task

In [ ]:
from init.patch_extraction import extractPatches25D

if not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi\\int8") and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi\\uint8"))):
    print("First, the ROI masks must be extracted from the OCT volumes.")
elif not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\patches\\2.5D")):
    extractPatches25D(IMAGES_PATH, patch_shape=(256,128), n_pos=12, n_neg=2, pos=1, neg=0)
else:
    print("Patches have already been extracted. To extract them again, please delete the folder that contains the extracted patches.")

### Models Tuning

*Run 1*
- Epochs: 100

Training

In [ ]:
from train import train_model

train_model(
    run_name="Run1",
    model_name="UNet",
    device="GPU",
    epochs=100,
    batch_size=32,
    learning_rate=2e-5,
    optimizer_name="Adam",
    momentum=0.999,
    weight_decay=0.0001,
    gradient_clipping=1.0,
    scheduler=False,
    number_of_classes=4,
    number_of_channels=1,
    fold_test=1,
    fold_val=2,
    tuning=True,
    patch_shape=(256,128), 
    n_pos=12, 
    n_neg=0, 
    pos=1, 
    neg=0,
    amp=True,
    assyncronous_patch_extraction=True,
    patch_dropping=True,
    drop_prob=0.75,
    patience=100
)

Plotting Logs

In [ ]:
from plot_logs import plot_logs

plot_logs(run_name="Run1")

**Results**
The first run. Due to a logic error, the best model was not saved and early stopping was not possible. However, it is interesting to understand the behavior of the U-Net in this conditions. While training loss slowly goes down to 0.26, the validation loss never crosses below the 0.39. Perhaps, a learning rate scheduler would be useful to implement in this conditions, to check if lower validation score is possible. Similarly, implementing this experiment with a lower learning rate may be useful to find a possible lower minimum error, with the expense of requiring a larger number of epochs in training. Also, it is important to mention that in this experiment, the data was obtained assycronously, unlike what was initially proposed, that recommended extracting random patches in every epoch. This later implementation must be compared against the assyncronous, to evaluate the necessaty of the patch extraction every epoch. It is important to note that the patch extraction is very time consuming, with the training process in an epoch taking as long as 15 minutes (without it, it takes 2 minutes).

![Training Error in Run1](./imgs/Run1_training_error.png)

Testing

In [ ]:
from test import test_model

test_model(
    fold_test=2,
    model_name="UNet",
    weights_name="Run1_UNet_best_model.pth",
    number_of_channels=1,
    number_of_classes=4,
    device_name="GPU",
    batch_size=1,
    save_images=True
)

**Results**

The Dice coefficient was calculated to each of the classes in each slice. In each slice, the number of positive voxels was recorded to allow the weighted average to calculate the real Dice coefficient of each volume or vendor. Then, the Dice coefficient of each class in each volume and in each vendor was calculated. The results can be seen in the CSV files in the folder results, organized per [slice](results/Run1_slice_dice.csv), [vendor](results/Run1_vendor_dice.csv), [volume](results/Run1_volume_dice.csv), and [class](results/Run1_class_dice.csv).

**Dice per vendor**

| Vendor     |   IRF   |   SRF   |   PED   |
| ---------- | ------- | ------- | ------- |
| Cirrus     |  0.458  |  0.307  |  0.390  |
| Spectralis |  0.644  |  0.725  |  0.494  |
| Topcon     |  0.356  |  0.475  |  0.403  |

**Dice per fluid**

|   IRF   |   SRF   |   PED   |
| ------- | ------- | ------- |
|  0.484  |  0.527  |  0.435  |

*Run 2*
- Epochs: 200

Training

In [ ]:
from train import train_model

train_model(
    run_name="Run1",
    model_name="UNet",
    device="GPU",
    epochs=200,
    batch_size=32,
    learning_rate=2e-5,
    optimizer_name="Adam",
    momentum=0.999,
    weight_decay=0.0001,
    gradient_clipping=1.0,
    scheduler=False,
    number_of_classes=4,
    number_of_channels=1,
    fold_test=1,
    fold_val=2,
    tuning=True,
    patch_shape=(256,128), 
    n_pos=12, 
    n_neg=0, 
    pos=1, 
    neg=0,
    amp=True,
    assyncronous_patch_extraction=True,
    patch_dropping=True,
    drop_prob=0.75,
    patience=100
)

Plotting Logs

In [ ]:
from plot_logs import plot_logs

plot_logs(run_name="Run2")

**Results**

In this run, the number of epochs was increased from 100 to 200, in order to verify whether it is possible to obtain better results in a longer run with the previous hyper parameters. The best model obtained in this training performs really similar to the previous, taking twice the same amount of time.

![Training Error in Run2](./imgs/Run2_training_error.png)

Testing

In [ ]:
from test import test_model

test_model(
    fold_test=2,
    model_name="UNet",
    weights_name="Run2_UNet_best_model.pth",
    number_of_channels=1,
    number_of_classes=4,
    device_name="GPU",
    batch_size=1,
    save_images=True
)

**Results**

The Dice coefficient was calculated to each of the classes in each slice. In each slice, the number of positive voxels was recorded to allow the weighted average to calculate the real Dice coefficient of each volume or vendor. Then, the Dice coefficient of each class in each volume and in each vendor was calculated. The results can be seen in the CSV files in the folder results, organized per [slice](results/Run2_slice_dice.csv), [vendor](results/Run2_vendor_dice.csv), [volume](results/Run2_volume_dice.csv), and [class](results/Run2_class_dice.csv).

**Dice per vendor**

| Vendor     |   IRF   |   SRF   |   PED   |
| ---------- | ------- | ------- | ------- |
| Cirrus     |  0.394  |  0.235  |  0.343  |
| Spectralis |  0.609  |  0.684  |  0.425  |
| Topcon     |  0.438  |  0.423  |  0.415  |

**Dice per fluid**

|   IRF   |   SRF   |   PED   |
| ------- | ------- | ------- |
|  0.477  |  0.472  |  0.385  |

*Run 3*
- Patch extraction was done every epoch without patch dropping, instead of once before the training

Training

In [ ]:
from train import train_model

train_model(
    run_name="Run3",
    model_name="UNet",
    device="GPU",
    epochs=100,
    batch_size=32,
    learning_rate=2e-5,
    optimizer_name="Adam",
    momentum=0.999,
    weight_decay=0.0001,
    gradient_clipping=1.0,
    scheduler=False,
    number_of_classes=4,
    number_of_channels=1,
    fold_test=1,
    fold_val=2,
    tuning=True,
    patch_shape=(256,128), 
    n_pos=12, 
    n_neg=0, 
    pos=1, 
    neg=0,
    amp=True,
    assyncronous_patch_extraction=False,
    patch_dropping=False,
    drop_prob=0.75,
    patience=100
)

Plotting Logs

In [ ]:
from plot_logs import plot_logs

plot_logs(run_name="Run3")

**Results**
The model was trained for a longer period of time, since it required patch extraction before training each epoch. Overall, an improvement in the Dice score was seen across all the vendors and classes, especially in the SRF class. However, it requires an increased amount of training time since larger batches are considered (without dropping patches) and the extraction of images makes the process longer.

![Training Error in Run3](./imgs/Run3_training_error.png)

Testing

In [ ]:
from test import test_model

test_model(
    fold_test=2,
    model_name="UNet",
    weights_name="Run3_UNet_best_model.pth",
    number_of_channels=1,
    number_of_classes=4,
    device_name="GPU",
    batch_size=1,
    save_images=True
)

**Results**

The Dice coefficient was calculated to each of the classes in each slice. In each slice, the number of positive voxels was recorded to allow the weighted average to calculate the real Dice coefficient of each volume or vendor. Then, the Dice coefficient of each class in each volume and in each vendor was calculated. The results can be seen in the CSV files in the folder results, organized per [slice](results/Run3_slice_dice.csv), [vendor](results/Run3_vendor_dice.csv), [volume](results/Run3_volume_dice.csv), and [class](results/Run3_class_dice.csv).

**Dice per vendor**

| Vendor     |   IRF   |   SRF   |   PED   |
| ---------- | ------- | ------- | ------- |
| Cirrus     |  0.456  |  0.296  |  0.301  |
| Spectralis |  0.684  |  0.763  |  0.489  |
| Topcon     |  0.322  |  0.525  |  0.468  |

**Dice per fluid**

|   IRF   |   SRF   |   PED   |
| ------- | ------- | ------- |
|  0.485  |  0.543  |  0.398  |

*Run 4*
- Syncronous patch extraction with patch dropping

Training

In [ ]:
from train import train_model

train_model(
    run_name="Run4",
    model_name="UNet",
    device="GPU",
    epochs=100,
    batch_size=32,
    learning_rate=2e-5,
    optimizer_name="Adam",
    momentum=0.999,
    weight_decay=0.0001,
    gradient_clipping=1.0,
    scheduler=False,
    number_of_classes=4,
    number_of_channels=1,
    fold_test=1,
    fold_val=2,
    tuning=True,
    patch_shape=(256,128), 
    n_pos=12, 
    n_neg=0, 
    pos=1, 
    neg=0,
    amp=True,
    assyncronous_patch_extraction=False,
    patch_dropping=True,
    drop_prob=0.75,
    patience=100
)

Plotting Logs

In [ ]:
from plot_logs import plot_logs

plot_logs(run_name="Run4")

**Results**
The results obtained regarding the loss during training and validation were really similar to those obtained with assyncronous patch extraction. This begs the question whether the improvements in last training were due to the randomization that patch extraction done in every epoch brings or due to the larger quantity of data available due to no employing patch dropout. One future experiment may be exactly that: re-run the first experiment without patch dropout. 

![Training Error in Run4](./imgs/Run4_training_error.png)

Testing

In [ ]:
from test import test_model

test_model(
    fold_test=2,
    model_name="UNet",
    weights_name="Run4_UNet_best_model.pth",
    number_of_channels=1,
    number_of_classes=4,
    device_name="GPU",
    batch_size=1,
    save_images=True
)

**Results**

The Dice coefficient was calculated to each of the classes in each slice. In each slice, the number of positive voxels was recorded to allow the weighted average to calculate the real Dice coefficient of each volume or vendor. Then, the Dice coefficient of each class in each volume and in each vendor was calculated. The results can be seen in the CSV files in the folder results, organized per [slice](results/Run4_slice_dice.csv), [vendor](results/Run4_vendor_dice.csv), [volume](results/Run4_volume_dice.csv), and [class](results/Run4_class_dice.csv).

**Dice per vendor**

| Vendor     |   IRF   |   SRF   |   PED   |
| ---------- | ------- | ------- | ------- |
| Cirrus     |  0.445  |  0.044  |  0.294  |
| Spectralis |  0.640  |  0.717  |  0.443  |
| Topcon     |  0.474  |  0.493  |  0.466  |

**Dice per fluid**

|   IRF   |   SRF   |   PED   |
| ------- | ------- | ------- |
|  0.516  |  0.401  |  0.374  |

*Run 5*
- Repetion of Run 1 without dropping patches  

Training

In [ ]:
from train import train_model

train_model(
    run_name="Run5",
    model_name="UNet",
    device="GPU",
    epochs=100,
    batch_size=32,
    learning_rate=2e-5,
    optimizer_name="Adam",
    momentum=0.999,
    weight_decay=0.0001,
    gradient_clipping=1.0,
    scheduler=False,
    number_of_classes=4,
    number_of_channels=1,
    fold_test=1,
    fold_val=2,
    tuning=True,
    patch_shape=(256,128), 
    n_pos=12, 
    n_neg=0, 
    pos=1, 
    neg=0,
    amp=True,
    assyncronous_patch_extraction=False,
    patch_dropping=False,
    drop_prob=0.75,
    patience=100
)

Plotting Logs

In [ ]:
from plot_logs import plot_logs

plot_logs(run_name="Run5")

**Results**

When compared with other runs, it is seen that the validation error becomes smaller than when runs use patch dropping. That happens because 50% of the loss used to train this network corresponds to the Dice loss in the background class. Since no patch dropping happens, the network gets better at segmenting the slices that do not present fluid, thus improving the loss values. However, this improved loss does not transmit to an improvement in segmentation performance, since values obtained when testing the network do not present an improvement in any metric when compared to those that do not present patch dropping, as in *Run1*.

![Training Error in Run5](./imgs/Run5_training_error.png)

Testing

In [ ]:
from test import test_model

test_model(
    fold_test=2,
    model_name="UNet",
    weights_name="Run5_UNet_best_model.pth",
    number_of_channels=1,
    number_of_classes=4,
    device_name="GPU",
    batch_size=1,
    save_images=True
)

**Results**

The Dice coefficient was calculated to each of the classes in each slice. In each slice, the number of positive voxels was recorded to allow the weighted average to calculate the real Dice coefficient of each volume or vendor. Then, the Dice coefficient of each class in each volume and in each vendor was calculated. The results can be seen in the CSV files in the folder results, organized per [slice](results/Run5_slice_dice.csv), [vendor](results/Run5_vendor_dice.csv), [volume](results/Run5_volume_dice.csv), and [class](results/Run5_class_dice.csv).

**Dice per vendor**

| Vendor     |   IRF   |   SRF   |   PED   |
| ---------- | ------- | ------- | ------- |
| Cirrus     |  0.350  |  0.201  |  0.323  |
| Spectralis |  0.675  |  0.716  |  0.423  |
| Topcon     |  0.371  |  0.290  |  0.518  |

**Dice per fluid**

|   IRF   |   SRF   |   PED   |
| ------- | ------- | ------- |
|  0.460  |  0.471  |  0.385  |

**Results**

The Dice coefficient was calculated to each of the classes in each slice. In each slice, the number of positive voxels was recorded to allow the weighted average to calculate the real Dice coefficient of each volume or vendor. Then, the Dice coefficient of each class in each volume and in each vendor was calculated. The results can be seen in the CSV files in the folder results, organized per [slice](results/Run5_slice_dice.csv), [vendor](results/Run5_vendor_dice.csv), [volume](results/Run5_volume_dice.csv), and [class](results/Run5_class_dice.csv).

**Dice per vendor**

| Vendor     |   IRF   |   SRF   |   PED   |
| ---------- | ------- | ------- | ------- |
| Cirrus     |  0.350  |  0.201  |  0.323  |
| Spectralis |  0.675  |  0.716  |  0.423  |
| Topcon     |  0.371  |  0.290  |  0.518  |

**Dice per fluid**

|   IRF   |   SRF   |   PED   |
| ------- | ------- | ------- |
|  0.460  |  0.471  |  0.385  |

*Run 6*
- Repetion of Run 1, dropping 50% of the non-pathological patches 

Training

In [ ]:
from train import train_model

train_model(
    run_name="Run6",
    model_name="UNet",
    device="GPU",
    epochs=100,
    batch_size=32,
    learning_rate=2e-5,
    optimizer_name="Adam",
    momentum=0.999,
    weight_decay=0.0001,
    gradient_clipping=1.0,
    scheduler=False,
    number_of_classes=4,
    number_of_channels=1,
    fold_test=1,
    fold_val=2,
    tuning=True,
    patch_shape=(256,128), 
    n_pos=12, 
    n_neg=0, 
    pos=1, 
    neg=0,
    amp=True,
    assyncronous_patch_extraction=True,
    patch_dropping=True,
    drop_prob=0.50,
    patience=100
)

Plotting Logs

In [ ]:
from plot_logs import plot_logs

plot_logs(run_name="Run6")

**Results**

Similarly to the last runs, a decreased loss value is obtained in models that handle more images with no pathology/no fluid to segment.

![Training Error in Run6](./imgs/Run6_training_error.png)

Testing

In [ ]:
from test import test_model

test_model(
    fold_test=2,
    model_name="UNet",
    weights_name="Run6_UNet_best_model.pth",
    number_of_channels=1,
    number_of_classes=4,
    device_name="GPU",
    batch_size=1,
    save_images=True
)

**Results**

The Dice coefficient was calculated to each of the classes in each slice. In each slice, the number of positive voxels was recorded to allow the weighted average to calculate the real Dice coefficient of each volume or vendor. Then, the Dice coefficient of each class in each volume and in each vendor was calculated. The results can be seen in the CSV files in the folder results, organized per [slice](results/Run6_slice_dice.csv), [vendor](results/Run6_vendor_dice.csv), [volume](results/Run6_volume_dice.csv), and [class](results/Run6_class_dice.csv).

The results obtained in this run present a slightly worse version than those obtained in Run7, not surpassing or obtaining any significantly improved results.

*Run 7*
- Repetion of Run 1, dropping 25% of the non-pathological patches 

Training

In [ ]:
from train import train_model

train_model(
    run_name="Run7",
    model_name="UNet",
    device="GPU",
    epochs=100,
    batch_size=32,
    learning_rate=2e-5,
    optimizer_name="Adam",
    momentum=0.999,
    weight_decay=0.0001,
    gradient_clipping=1.0,
    scheduler=False,
    number_of_classes=4,
    number_of_channels=1,
    fold_test=1,
    fold_val=2,
    tuning=True,
    patch_shape=(256,128), 
    n_pos=12, 
    n_neg=0, 
    pos=1, 
    neg=0,
    amp=True,
    assyncronous_patch_extraction=True,
    patch_dropping=True,
    drop_prob=0.25,
    patience=100
)

Plotting Logs

In [ ]:
from plot_logs import plot_logs

plot_logs(run_name="Run7")

**Results**

The results in training achieved a better loss due to a higher quantity of non-pathological scans when compared to the previous models. 

![Training Error in Run7](./imgs/Run7_training_error.png)

Testing

In [ ]:
from test import test_model

test_model(
    fold_test=2,
    model_name="UNet",
    weights_name="Run7_UNet_best_model.pth",
    number_of_channels=1,
    number_of_classes=4,
    device_name="GPU",
    batch_size=1,
    save_images=True
)

**Results**

The Dice coefficient was calculated to each of the classes in each slice. In each slice, the number of positive voxels was recorded to allow the weighted average to calculate the real Dice coefficient of each volume or vendor. Then, the Dice coefficient of each class in each volume and in each vendor was calculated. The results can be seen in the CSV files in the folder results, organized per [slice](results/Run6_slice_dice.csv), [vendor](results/Run6_vendor_dice.csv), [volume](results/Run6_volume_dice.csv), and [class](results/Run6_class_dice.csv).

So far, Run7 showed the best Dice performance, as it achieved some of the best results among the runs, while also keeping up with the best runs across the rest, therefore showcasing the most consistency.

*Run 8*
- Repetion of Run 1, dropping 100% of the non-pathological patches 

Training

In [ ]:
from train import train_model

train_model(
    run_name="Run8",
    model_name="UNet",
    device="GPU",
    epochs=100,
    batch_size=32,
    learning_rate=2e-5,
    optimizer_name="Adam",
    momentum=0.999,
    weight_decay=0.0001,
    gradient_clipping=1.0,
    scheduler=False,
    number_of_classes=4,
    number_of_channels=1,
    fold_test=1,
    fold_val=2,
    tuning=True,
    patch_shape=(256,128), 
    n_pos=12, 
    n_neg=0, 
    pos=1, 
    neg=0,
    amp=True,
    assyncronous_patch_extraction=True,
    patch_dropping=True,
    drop_prob=1.,
    patience=100
)

Plotting Logs

In [ ]:
from plot_logs import plot_logs

plot_logs(run_name="Run8")

**Results**

The results in training achieved a significantly higher loss due to a higher quantity of non-pathological scans when compared to the previous models. 

![Training Error in Run8](./imgs/Run8_training_error.png)

Testing

In [ ]:
from test import test_model

test_model(
    fold_test=2,
    model_name="UNet",
    weights_name="Run8_UNet_best_model.pth",
    number_of_channels=1,
    number_of_classes=4,
    device_name="GPU",
    batch_size=1,
    save_images=True
)

**Results**

The Dice coefficient was calculated to each of the classes in each slice. In each slice, the number of positive voxels was recorded to allow the weighted average to calculate the real Dice coefficient of each volume or vendor. Then, the Dice coefficient of each class in each volume and in each vendor was calculated. The results can be seen in the CSV files in the folder results, organized per [slice](results/Run8_slice_dice.csv), [vendor](results/Run8_vendor_dice.csv), [volume](results/Run8_volume_dice.csv), and [class](results/Run8_class_dice.csv).

So far, Run8 achieved outstanding performances on Cirrus when segmenting IRF and PED and good performances on Topcon in IRF and SRF segmentation, but on the other classes was not so good. However, the performance in Spectralis was worse than in the remaining models.

### **Overall Testing Results**

**Runs**
- *Run 1*: 100 Epochs, assyncronous patch extraction, and 75% drop percentage
- *Run 2*: 200 Epochs, assyncronous patch extraction, and 75% drop percentage
- *Run 3*: 100 Epochs, syncronous patch extraction, and 0% drop percentage
- *Run 4*: 100 Epochs, syncronous patch extraction, and 75% drop percentage
- *Run 5*: 100 Epochs, assyncronous patch extraction, and 0% drop percentage
- *Run 6*: 100 Epochs, assyncronous patch extraction, and 50% drop percentage (REPEAT)
- *Run 7*: 100 Epochs, assyncronous patch extraction, and 25% drop percentage
- *Run 8*: 100 Epochs, assyncronous patch extraction, and 100% drop percentage


**Dice per vendor**

|          |              |     **IRF**    |              |             |    **SRF**     |              |             |    **PED**     |              |
| :------: | :----------: | :------------: | :----------: | :---------: | :------------: | :----------: | :---------: | :------------: | :----------: |
| **Run**  |  **Cirrus**  | **Spectralis** |  **Topcon**  |  **Cirrus** | **Spectralis** |  **Topcon**  |  **Cirrus** | **Spectralis** |  **Topcon**  |
|   Run1   |    0.235     |     0.515      |    0.401     |  **0.176**  |     0.706      |  **0.100**   |    0.191    |     0.470      |    0.123     |
|   Run2   |    0.156     |     0.463      |    0.462     |    0.106    |     0.660      |    0.014     |    0.177    |     0.411      |    0.139     |
|   Run3   |    0.230     |     0.582      |    0.371     |    0.103    |   **0.751**    |    0.004     |    0.217    |     0.491      |    0.226     |
|   Run4   |    0.191     |     0.513      |  **0.530**   |    0.105    |     0.690      |    0.075     |    0.193    |     0.436      |  **0.233**   |
|   Run5   |    0.133     |   **0.592**    |    0.274     |    0.069    |     0.703      |    0.014     |    0.183    |     0.428      |    0.229     |
|   Run6   |     NaN      |      NaN       |     NaN      |     NaN     |      NaN       |     NaN      |     NaN     |      NaN       |     NaN      |
|   Run7   |    0.312     |     0.559      |    0.295     |    0.045    |     0.728      |    0.038     |  **0.240**  |   **0.518**    |    0.197     |
|   Run8   |  **0.354**   |     0.500      |    0.431     |    0.059    |     0.670      |    0.039     |    0.177    |     0.319      |    0.052     |

**Dice per fluid**

 Run  |   IRF   |   SRF   |   PED   |
 :--: | :-----: | :-----: | :-----: |
 Run1 |  0.355  |**0.371**|  0.223  |
 Run2 |  0.272  |  0.218  |  0.215  |
 Run3 |  0.363  |  0.275  |  0.306  |
 Run4 |  0.343  |  0.191  |  0.278  |
 Run5 |  0.259  |  0.189  |  0.251  |
 Run6 |   NaN   |   NaN   |   NaN   |
 Run7 |  0.399  |  0.190  |**0.308**|
 Run8 |**0.427**|  0.171  |  0.142  |

In [ ]:
from train import train_model

train_model(
    run_name="Run9",
    model_name="UNet",
    device="GPU",
    epochs=100,
    batch_size=32,
    learning_rate=2e-5,
    optimizer_name="Adam",
    momentum=0.999,
    weight_decay=0.0001,
    gradient_clipping=1.0,
    scheduler=False,
    number_of_classes=4,
    number_of_channels=1,
    fold_test=1,
    fold_val=2,
    tuning=True,
    patch_shape=(256,128), 
    n_pos=12, 
    n_neg=0, 
    pos=1, 
    neg=0,
    amp=True,
    assyncronous_patch_extraction=True,
    patch_dropping=True,
    drop_prob=1.,
    patience=100
)

Training on cuda.


INFO: Network
	1 input channels
	4 output channels (classes)

INFO: Starting training:
        Epochs:          100
        Batch size:      32
        Learning rate:   2e-05
        Device:          cuda
        Mixed Precision: True
    


Extracting patches
Extracting Training Patches


## Generation

### Train-Test Split

Create Train-Test Split for the Intermediate Slice Generation Task

In [ ]:
from init.folds_split import k_fold_split_generation

k = 5

if not (isfile(path="splits/generation_train_splits.csv") or isfile(path="splits/generation_test_splits.csv")):
    k_fold_split_generation(k=k, folders_path=RETOUCH_PATH)
else:
    print("Split already exists. To create a new one please delete the old files.")

### Images Saving

OCT Volumes Reading and Saving for Generation Task

Note: To make the generation task independent from the segmentation task, the images used for segmentation are being saved again in a different folder. For memory concerns, please adjust the code to reuse those saved previously.

In [ ]:
from init.read_oct import save_generation_oct_as_tiff

if not ((exists(IMAGES_PATH + "\\OCT_images\\generation\\int32")) and (exists(IMAGES_PATH + "\\OCT_images\\generation\\uint8"))):
    save_generation_oct_as_tiff(oct_folder=RETOUCH_PATH, save_folder=IMAGES_PATH)
else:
    print("Images have already been extracted. To extract them again, please delete the folder with the images.")